In [1]:
import tensorflow as tf
import random
import os
import sys
import numpy as np
# import torch
import logging
import argparse
import shutil
import cv2
import time
import threading
import multiprocessing
from pathlib import Path
from tqdm import tqdm
import pandas as pd
from matplotlib import pyplot as plt
# from icecream import ic
from collections import defaultdict
import re

In [4]:
import tensorflow_datasets as tfds

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(device_type='GPU', devices=gpus[0])

In [ ]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

2022-03-05 07:43:52.766539: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".
2022-03-05 07:46:58.182725: E tensorflow/core/platform/cloud/curl_http_request.cc:614] The transmission  of request 0x5636ec801440 (URI: https://www.googleapis.com/storage/v1/b/tfds-data/o/dataset_info%2Fimdb_reviews%2Fplain_text%2F1.0.0?fields=size%2Cgeneration%2Cupdated) has been stuck at 0 of 0 bytes for 61 seconds and will be aborted. CURL timing information: lookup time: 0.021579 (No error), connect time: 0 (No error), pre-transfer time: 0 (No error), start-transfer time: 0 (No error)
2022-03-05 07:48:04.932612: E tenso

2022-03-05 07:53:27.972488: E tensorflow/core/platform/cloud/curl_http_request.cc:614] The transmission  of request 0x5636ebf69da0 (URI: https://www.googleapis.com/storage/v1/b/tfds-data/o/datasets%2Fimdb_reviews%2Fplain_text%2F1.0.0?fields=size%2Cgeneration%2Cupdated) has been stuck at 0 of 0 bytes for 61 seconds and will be aborted. CURL timing information: lookup time: 0.022174 (No error), connect time: 0 (No error), pre-transfer time: 0 (No error), start-transfer time: 0 (No error)
2022-03-05 07:54:29.294202: E tensorflow/core/platform/cloud/curl_http_request.cc:614] The transmission  of request 0x5636ebf69da0 (URI: https://www.googleapis.com/storage/v1/b/tfds-data/o/datasets%2Fimdb_reviews%2Fplain_text%2F1.0.0?fields=size%2Cgeneration%2Cupdated) has been stuck at 0 of 0 bytes for 61 seconds and will be aborted. CURL timing information: lookup time: 0.031564 (No error), connect time: 0 (No error), pre-transfer time: 0 (No error), start-transfer time: 0 (No error)


In [6]:
type(imdb)
imdb

{'train': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>,
 'test': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>,
 'unsupervised': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>}

In [7]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset.
    This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='/root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0',
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
        'train': <SplitInfo num

In [8]:
train_data, test_data = imdb['train'], imdb['test']

In [9]:
train_data, test_data

(<PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>,
 <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>)

In [10]:
train_data.take(1)

<TakeDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [11]:
training_sentences = []
training_labels = []
for s, l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())
    # print(s)
    # print(l)

In [12]:
training_sentences = np.array(training_sentences)
training_labels = np.array(training_labels)
training_sentences.shape, training_labels.shape

((25000,), (25000,))

In [13]:
test_sentences = []
test_labels = []
for s, l in test_data:
    test_sentences.append(s.numpy().decode('utf8'))
    test_labels.append(l.numpy())
    # print(s)
    # print(l)

In [14]:
test_sentences = np.array(test_sentences)
test_labels = np.array(test_labels)
test_sentences.shape,test_labels.shape

((25000,), (25000,))

In [15]:
training_labels

array([0, 0, 0, ..., 0, 0, 1])

In [16]:
test_labels

array([1, 1, 0, ..., 0, 1, 1])

### 1

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [18]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(np.concatenate((training_sentences, test_sentences)))
tokenizer.word_index
word_index = tokenizer.word_index
len(word_index),type(word_index)

(124252, dict)

In [19]:
from itertools import islice

In [20]:
list(islice(tokenizer.word_index.items(), 0, 10))

[('the', 1),
 ('and', 2),
 ('a', 3),
 ('of', 4),
 ('to', 5),
 ('is', 6),
 ('br', 7),
 ('in', 8),
 ('it', 9),
 ('i', 10)]

In [21]:
training_sequences_id = tokenizer.texts_to_sequences(training_sentences)
test_sequences_id = tokenizer.texts_to_sequences(test_sentences)

In [22]:
training_padded = pad_sequences(training_sequences_id, padding='post')
test_padded = pad_sequences(test_sequences_id, padding='post')
training_padded.shape, test_padded.shape, len(training_labels), len(test_labels)

((25000, 2142), (25000, 2209), 25000, 25000)

In [23]:
type(training_labels)

numpy.ndarray

### train

In [3]:
model = tf.keras.Sequential(
    [
      tf.keras.layers.Embedding(len(word_index), 16), 
      tf.keras.layers.GlobalAveragePooling1D(),
      tf.keras.layers.Dense(24, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid'),
    ]
)
model.summary()

NameError: name 'word_index' is not defined

In [25]:
model = tf.keras.Sequential(
    [
      tf.keras.layers.Embedding(10001, 16), 
      tf.keras.layers.GlobalAveragePooling1D(),
      tf.keras.layers.Dense(24, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid'),
    ]
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 16)          160016    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 24)                408       
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160,449
Trainable params: 160,449
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(
    loss='binary_crossentropy',
    optimizer = 'adam', 
    metrics=['accuracy']
)

In [27]:
model.fit(training_padded, training_labels, epochs=10, validation_data=(test_padded, test_labels), verbose=1)

Epoch 1/10
 38/782 [>.............................] - ETA: 3s - loss: 0.6933 - accuracy: 0.4844

2022-02-28 08:38:21.913289: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


782/782 [==============================] - 7s 7ms/step - loss: 0.6818 - accuracy: 0.5811 - val_loss: 0.6339 - val_accuracy: 0.7810
Epoch 2/10
782/782 [==============================] - 5s 7ms/step - loss: 0.5036 - accuracy: 0.8063 - val_loss: 0.4415 - val_accuracy: 0.8083
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.3531 - accuracy: 0.8692 - val_loss: 0.3451 - val_accuracy: 0.8671
Epoch 4/10
782/782 [==============================] - 6s 7ms/step - loss: 0.2951 - accuracy: 0.8882 - val_loss: 0.3148 - val_accuracy: 0.8760
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 0.2634 - accuracy: 0.9005 - val_loss: 0.3213 - val_accuracy: 0.8663
Epoch 6/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2436 - accuracy: 0.9080 - val_loss: 0.3160 - val_accuracy: 0.8668
Epoch 7/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2237 - accuracy: 0.9167 - val_loss: 0.2819 - val_accuracy: 0.8896
Epoch 8/10
782/78

In [28]:
# training_sentences
x = tf.random.normal([5, 2000])
y = model(x)
y

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[0.9997123 ],
       [0.99969983],
       [0.9995697 ],
       [0.99956626],
       [0.99944013]], dtype=float32)>

In [29]:
len(word_index)

124252

In [30]:
list(islice(tokenizer.word_index.items(), 124250, 124255))

[("regurgitation's", 124251), ("hallowe'en", 124252)]

### gru

In [3]:
# gru = tf.keras.layers.GRU(4)
tf.keras.layers.GRU?

In [32]:
inputs = tf.random.normal([32,10,8])

In [2]:
gru = tf.keras.layers.GRU(4,return_state=True)
inputs = tf.random.normal([32,10,8])
st, yt = gru(inputs)
yt.shape, st.shape # yt, no st

2022-03-05 07:55:55.378179: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-05 07:55:56.445750: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46713 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:89:00.0, compute capability: 8.6
2022-03-05 07:55:56.446708: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46713 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2022-03-05 07:55:58.196084: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201


(TensorShape([32, 4]), TensorShape([32, 4]))

In [4]:
gru = tf.keras.layers.GRU(4,return_state=True, return_sequences=True)
inputs = tf.random.normal([32,10,8])
st, yt = gru(inputs)
yt.shape, st.shape # yt, no st

(TensorShape([32, 4]), TensorShape([32, 10, 4]))

In [ ]:
gru = tf.keras.layers.GRU(4,return_sequences=True)
inputs = tf.random.normal([32,10,8])
st = gru(inputs)
st.shape # yt, no st

In [15]:
model2 = tf.keras.Sequential(
    [
      tf.keras.layers.Embedding(10001, 16),
      tf.keras.layers.GRU(8),
      tf.keras.layers.Dense(24, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
    ]
)
model2.compile(
    loss='binary_crossentropy',
    optimizer = 'adam', 
    metrics=['accuracy']
)
model2.summary()
x = tf.random.normal([3,10])
# x = tf.zeros([3,10])
y = model2(x)
y.shape
y

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, None, 16)          160016    
                                                                 
 gru_10 (GRU)                (None, 8)                 624       
                                                                 
 dense_14 (Dense)            (None, 24)                216       
                                                                 
 dense_15 (Dense)            (None, 1)                 25        
                                                                 
Total params: 160,881
Trainable params: 160,881
Non-trainable params: 0
_________________________________________________________________


<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.49876103],
       [0.49799582],
       [0.49867564]], dtype=float32)>

In [ ]:
model2.compile(
    loss='binary_crossentropy',
    optimizer = 'adam', 
    metrics=['accuracy']
)

In [ ]:
x = tf.random.normal([5, 2000])
y = model2(x)
y

In [ ]:
model2.fit(training_padded, training_labels, epochs=20, validation_data=(test_padded, test_labels), verbose=1)

### gru2

In [ ]:
model3 = tf.keras.Sequential(
    [
      tf.keras.layers.Embedding(10001, 16),
      tf.keras.layers.GRU(32, return_sequences=True),
      # tf.keras.layers.GRU(32, return_sequences=True),
      # tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(16, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
    ]
)
model3.summary()
x = tf.random.normal([5, 2000])
y = model3(x)
y.shape

In [ ]:
model3.compile(
    loss='binary_crossentropy',
    optimizer = 'adam', 
    metrics=['accuracy']
)

In [ ]:
model3.fit(training_padded, training_labels, epochs=10)

### bidirection gru

In [ ]:
model4 = tf.keras.Sequential(
    [
      tf.keras.layers.Embedding(10001, 16),
      tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True)),
      tf.keras.layers.Dense(16, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
    ]
)
model4.summary()
x = tf.random.normal([5, 2000])
y = model4(x)
y.shape